In [5]:
# Importacion de galerias
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import folium
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.metrics import pairwise_distances

In [16]:
# Cargar el archivo .pkl como un DataFrame
df = pd.read_pickle('../Datasets/Yelp/business.pkl') # `test01-440321.test001.business`
df #PK business_id  

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#tratamiento del DB para el desarrollo del ML
df = df.iloc[:, :14] #recorto columnas duplicadas
df = df.iloc[:500] #me quedo con las primeras 500 lineas para optimizar pruebas
df = df[['business_id','name', 'latitude','longitude','stars']] #Solo me quedo con las columnas intervenientes en el ML
df

,business_id,name,latitude,longitude,stars
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",34.426679,-119.711197,5.0
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,38.551126,-90.335695,3.0
2,tUFrWirKiKi_TAnsVWINQQ,Target,32.223236,-110.880452,3.5
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,39.955505,-75.155564,4.0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,40.338183,-75.471659,4.5
...,...,...,...,...,...
495,rrdSkNz4YG6eDgOq3ov0IQ,Thoroughbred Nissan,32.207596,-110.881629,3.5
496,RsTF2skxLcp6qo9FdnA5PA,Pizza Hut,40.21786,-74.926634,1.5
497,WZpCnvSQ-YGukju9r4Gd_A,Morales Landscapers,32.268074,-110.99239,4.0
498,PXI7kvSV-NvWcqbI9jlmtg,Best Western Cedar Park Inn,53.488591,-113.493509,3.0


## DBScan

In [19]:
# Creo un array con las coordenadas
coords = df[['latitude', 'longitude']].values

dbscan = DBSCAN(eps=0.09, min_samples=10) #Considero una cantidad de 10 restaurantes distanciados a un radio menor a 0.09 para considerarlo un polo

# Entreno el modelo con las coordenadas
dbscan.fit(coords) #clusters = dbscan.fit_predict(coords)
clusters = dbscan.fit_predict(coords)

# Agrega la columna de clusters al DataFrame
df['cluster'] = clusters

#### Comprobaciones

In [ ]:
print(df['cluster'].unique())
cluster_qty = len(df['cluster'].unique())-1 #el polo (-1) se considera ruido para el algoritmo
print(f"Cantidad de polos gastronomicos: {cluster_qty}")

[ 0  1  2  3 -1  4  5  6  7  8  9 10 11 12]
Cantidad de polos gastronomicos: 13


vemos que se generaron 13 cluster (polos gastronomicos), al trabajar con un dataset recortado y al trabajar con hiperparametros seleccionados para la velocidad consideramos que es correcta la generacion de 13 cluster

In [25]:
df

,business_id,name,latitude,longitude,stars,cluster
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",34.426679,-119.711197,5.0,0
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,38.551126,-90.335695,3.0,1
2,tUFrWirKiKi_TAnsVWINQQ,Target,32.223236,-110.880452,3.5,2
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,39.955505,-75.155564,4.0,3
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,40.338183,-75.471659,4.5,-1
...,...,...,...,...,...,...
495,rrdSkNz4YG6eDgOq3ov0IQ,Thoroughbred Nissan,32.207596,-110.881629,3.5,2
496,RsTF2skxLcp6qo9FdnA5PA,Pizza Hut,40.21786,-74.926634,1.5,3
497,WZpCnvSQ-YGukju9r4Gd_A,Morales Landscapers,32.268074,-110.99239,4.0,2
498,PXI7kvSV-NvWcqbI9jlmtg,Best Western Cedar Park Inn,53.488591,-113.493509,3.0,8


Como se puede observar se agregó la columna cluster que indica a que polo gastronomico pertenece cada restaurant.
El algoritmo asigna el cluster (-1) a aquellos restaurants que estan alejados de los polos

### Mapa con Folium

In [21]:
# defino una lista de colores para asignar a cada cluster
marker_colors = [
    'darkgreen',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'green',
    'beige',
    'red',
    'orange', #canada city: edmonton
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black']

In [22]:
#Creacion del mapa
map_center = [32.2437, -89.736] #Centro el mapa en EEUU
mapa = folium.Map(location=map_center, zoom_start=4)

for cluster_id in df['cluster'].unique():
  if cluster_id != -1:  # Ignora los puntos ruido (cluster = -1)
    cluster_df = df[df['cluster'] == cluster_id]
    for _, row in cluster_df.iterrows():
      folium.CircleMarker(
          location=[row['latitude'], row['longitude']],
          radius=15,
          icon=folium.Icon(color=marker_colors[cluster_id]),  # Cambiar el color según el cluster_id
          fill=True,
          fill_color=marker_colors[cluster_id],  # Cambiar el color según el cluster_id
          fill_opacity=.3
      ).add_to(mapa)

In [23]:
mapa

En el mapa se puede ver que los restaurantes que estan cercanos a un epsilon de distancia y que estan rodeados de un grupo minimo de 10 estan coloreados del mismo color (fue agrupado por el algoritmo al mismo cluster), dando otra comprobacion de que el algoritmo esta funcionando correctamente para esos hiperparametros (epsilon=0.09 y samples=10)

## Carga de BD de Google, en esta instancia local las BD estan separadas en 2 archivos

In [35]:
# Crear el DataFrame
dfgoogle = pd.read_parquet("..\Datasets\Yelp\metadata-sitios_restaurantes.parquet") #`test01-440321.test001.restaurante`
dfgoogle # PK gmap_id

dfgoogle.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1351492 entries, 0 to 212013
Data columns (total 25 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   name               1351492 non-null  object 
 1   gmap_id            1351492 non-null  object 
 2   latitude           1351492 non-null  float64
 3   longitude          1351492 non-null  float64
 4   category           1351492 non-null  object 
 5   avg_rating         1351492 non-null  float64
 6   num_of_reviews     1351492 non-null  int64  
 7   price              650026 non-null   object 
 8   day                1329391 non-null  object 
 9   hours              1329391 non-null  object 
 10  Service options    1310659 non-null  object 
 11  Accessibility      931656 non-null   object 
 12  Offerings          1015927 non-null  object 
 13  Amenities          1051418 non-null  object 
 14  Atmosphere         909195 non-null   object 
 15  Health & safety    373665 non-null   o

In [38]:
dfgoogle.iloc[:1000]

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,day,hours,...,Health & safety,Popular for,Dining options,Crowd,Payments,Highlights,Planning,From the business,Health and safety,Recycling
0,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,None,Thursday,6:30AM–6PM,...,None,None,None,None,None,None,None,None,None,None
0,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,None,Friday,6:30AM–6PM,...,None,None,None,None,None,None,None,None,None,None
0,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,None,Saturday,6:30AM–6PM,...,None,None,None,None,None,None,None,None,None,None
0,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,None,Sunday,7AM–12PM,...,None,None,None,None,None,None,None,None,None,None
0,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18,None,Monday,Closed,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Restaurant Josephine,0x809b053254d9dfab:0xb1b11e804211e0ea,38.898093,-121.074669,Restaurant,4.7,17,None,Saturday,5–10PM,...,None,None,None,None,None,None,None,None,None,None
164,Restaurant Josephine,0x809b053254d9dfab:0xb1b11e804211e0ea,38.898093,-121.074669,Restaurant,4.7,17,None,Sunday,Closed,...,None,None,None,None,None,None,None,None,None,None
164,Restaurant Josephine,0x809b053254d9dfab:0xb1b11e804211e0ea,38.898093,-121.074669,Restaurant,4.7,17,None,Monday,Closed,...,None,None,None,None,None,None,None,None,None,None
164,Restaurant Josephine,0x809b053254d9dfab:0xb1b11e804211e0ea,38.898093,-121.074669,Restaurant,4.7,17,None,Tuesday,5–9PM,...,None,None,None,None,None,None,None,None,None,None


In [51]:
#tratamiento del DB para el desarrollo del ML
dfgoogle = dfgoogle.iloc[:3310] #me quedo con las primeras 3310 lineas para optimizar pruebas
dfgoogle = dfgoogle[['gmap_id','name', 'latitude','longitude','avg_rating']] #Solo me quedo con las columnas intervenientes en el ML
dfgoogle = dfgoogle.drop_duplicates(subset=['name'], keep='first') # sacamos lineas duplicadas
dfgoogle

,gmap_id,name,latitude,longitude,avg_rating
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,34.058092,-118.292130,4.4
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,33.916402,-118.010855,4.5
2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",41.616079,-93.865487,4.7
3,0x89c6c89efcaed69d:0xded973f6033e7dba,Vivi Bubble Tea,39.940293,-75.150923,4.0
4,0x7c00456eecad3111:0x8217f9600c51f33,Hale Pops,21.637796,-157.920714,4.4
...,...,...,...,...,...
527,0x89c24544dcba20d9:0x81b7791c2791231e,月花坊 Moon Flower House,40.637223,-74.011716,2.9
528,0x89c25ab649a37525:0x91f6c1bcc70b6a91,Rax's Pizza,40.646968,-74.008054,4.1
529,0x88e879b906e0ef23:0xa598ddc276ff5bbf,Mama B's Diner,29.090498,-82.246866,5.0
530,0x54910a92e8fd3899:0xb7502fbd3e94663e,On Point Pizza,47.080820,-122.844859,4.9


#### Para asignar los restaurant de la nueva BD a los cluster ya generados por la anterior desarrollamos una funcion que calcula la distancia de cada nuevo punto con los anteriores y en base a un epsilon y un sample nuevo define si pertenecen a un cluster o no

In [52]:
def predict_dbscan_point(dbscan_model, X_train, new_point, epsilon, min_samples):
        # Calcular distancias del nuevo punto a todos los puntos en X_train
    distances = pairwise_distances([new_point], X_train).flatten()
    
    # Encontrar puntos dentro del radio epsilon
    neighbors_in_radius = np.where(distances <= epsilon)[0]
    
    # Verificar si cumple con la densidad mínima
    if len(neighbors_in_radius) >= min_samples:
        # Etiqueta de los vecinos en el radio epsilon
        neighbor_labels = dbscan_model.labels_[neighbors_in_radius]
                
        # Filtrar etiquetas de ruido (-1) y obtener la etiqueta de cluster más común
        cluster_labels = neighbor_labels[neighbor_labels != -1]
        
        if len(cluster_labels) > 0:
            # Devolver el cluster más común entre los vecinos
            return np.bincount(cluster_labels).argmax()
    
    # Si no cumple los criterios, clasificar como ruido (-1)
    return -1

In [53]:
# Creo un array con las coordenadas de los nuevos puntos
nuevos_coords = dfgoogle[['latitude', 'longitude']].values

In [55]:
#Clasifico los nuevos puntos
predictions = [predict_dbscan_point(dbscan, coords, point, 1 , 10) for point in nuevos_coords]
print("Predicción de clusters para nuevos puntos:", predictions)

Predicción de clusters para nuevos puntos: [-1, -1, -1, np.int64(3), -1, -1, np.int64(6), -1, -1, -1, -1, -1, -1, -1, np.int64(7), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, np.int64(1), -1, -1, -1, np.int64(5), -1, -1, -1, -1, -1, np.int64(6), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, np.int64(5), -1, -1, -1, -1, -1, -1, -1, np.int64(3), np.int64(3), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, np.int64(3), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, np.int64(3), -1, -1, -1, -1, -1, -1, -1, np.int64(3), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, np.int64(3), -1, -1, -1, -1, -1, -1, -1, np.int64(7), np.int64(1), np.int64(11), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, np.int64(3), -1, -1, -1, np.int64(12), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 

In [56]:
#el resultado de la funcion lo agrego a la BD
dfgoogle['cluster'] = predictions

#### Comprobaciones

In [62]:
print(dfgoogle['cluster'].unique())
cluster_qty = len(dfgoogle['cluster'].unique())-1
print(f"Cantidad de polos gastronomicos: {cluster_qty}")

[-1  3  6  7  1  5 11 12  9]
Cantidad de polos gastronomicos: 8


## Mapa de los restaurante de Google asignando un color a cada cluster con Folium

In [63]:
#Creacion del mapa
map_center = [32.2437, -89.736] #Centro el mapa en EEUU
mapa = folium.Map(location=map_center, zoom_start=4)

for cluster_id in dfgoogle['cluster'].unique():
  if cluster_id != -1 and cluster_id < 16:  # Ignora los puntos ruido (cluster = -1) y mayores a 16 por colores
    cluster_df = dfgoogle[dfgoogle['cluster'] == cluster_id]
    for _, row in cluster_df.iterrows():
      folium.CircleMarker(
          location=[row['latitude'], row['longitude']],
          radius=15,
          icon=folium.Icon(color=marker_colors[cluster_id]),  # Cambiar el color según el cluster_id
          fill=True,
          fill_color=marker_colors[cluster_id],  # Cambiar el color según el cluster_id
          fill_opacity=.3
      ).add_to(mapa)

In [64]:
mapa